In [16]:
# Import all of the things you need to import!
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from nltk.stem.porter import PorterStemmer

pd.options.display.max_columns = 30
%matplotlib inline

# Homework 14 (or so): TF-IDF text analysis and clustering

Hooray, we kind of figured out how text analysis works! Some of it is still magic, but at least the **TF** and **IDF** parts make a little sense. Kind of. Somewhat.

No, just kidding, we're *professionals* now.

## Investigating the Congressional Record

The [Congressional Record](https://en.wikipedia.org/wiki/Congressional_Record) is more or less what happened in Congress every single day. Speeches and all that. A good large source of text data, maybe?

Let's pretend it's totally secret but we just got it leaked to us in a data dump, and we need to check it out. It was leaked from [this page here](http://www.cs.cornell.edu/home/llee/data/convote.html).

In [2]:
# If you'd like to download it through the command line...
!curl -O http://www.cs.cornell.edu/home/llee/data/convote/convote_v1.1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9607k  100 9607k    0     0  5608k      0  0:00:01  0:00:01 --:--:-- 5611k


In [3]:
# And then extract it through the command line...
!tar -zxf convote_v1.1.tar.gz

You can explore the files if you'd like, but we're going to get the ones from `convote_v1.1/data_stage_one/development_set/`. It's a bunch of text files.

In [2]:
# glob finds files matching a certain filename pattern
import glob

# Give me all the text files
paths = glob.glob('convote_v1.1/data_stage_one/development_set/*')
paths[:5]

['convote_v1.1/data_stage_one/development_set/052_400011_0327014_DON.txt',
 'convote_v1.1/data_stage_one/development_set/052_400011_0327025_DON.txt',
 'convote_v1.1/data_stage_one/development_set/052_400011_0327044_DON.txt',
 'convote_v1.1/data_stage_one/development_set/052_400011_0327046_DON.txt',
 'convote_v1.1/data_stage_one/development_set/052_400011_1479036_DON.txt']

In [3]:
len(paths)

702

So great, we have 702 of them. Now let's import them.

In [4]:
speeches = []
for path in paths:
    with open(path) as speech_file:
        speech = {
            'pathname': path,
            'filename': path.split('/')[-1],
            'content': speech_file.read()
        }
    speeches.append(speech)
speeches_df = pd.DataFrame(speeches)
speeches_df.head()

,content,filename,pathname
0,"mr. chairman , i thank the gentlewoman for yie...",052_400011_0327014_DON.txt,convote_v1.1/data_stage_one/development_set/05...
1,"mr. chairman , i want to thank my good friend ...",052_400011_0327025_DON.txt,convote_v1.1/data_stage_one/development_set/05...
2,"mr. chairman , i rise to make two fundamental ...",052_400011_0327044_DON.txt,convote_v1.1/data_stage_one/development_set/05...
3,"mr. chairman , reclaiming my time , let me mak...",052_400011_0327046_DON.txt,convote_v1.1/data_stage_one/development_set/05...
4,"mr. chairman , i thank my distinguished collea...",052_400011_1479036_DON.txt,convote_v1.1/data_stage_one/development_set/05...


In class we had the `texts` variable. For the homework can just do `speeches_df['content']` to get the same sort of list of stuff.

**Take a look at the contents of the first 5 speeches**

In [5]:
speeches_df['content'].head()

0    mr. chairman , i thank the gentlewoman for yie...
1    mr. chairman , i want to thank my good friend ...
2    mr. chairman , i rise to make two fundamental ...
3    mr. chairman , reclaiming my time , let me mak...
4    mr. chairman , i thank my distinguished collea...
Name: content, dtype: object

# Doing our analysis

Use the `sklearn` package and a plain boring `CountVectorizer` to get a list of all of the tokens used in the speeches. If it won't list them all, that's ok! Make a dataframe with those terms as columns.

**Be sure to include English-language stopwords**

In [6]:
speeches = speeches_df['content'].head(100)

In [7]:
list_of_speeches = speeches.tolist()
list_of_speeches

["mr. chairman , i thank the gentlewoman for yielding me this time . \nmy good colleague from california raised the exact and critical point . \nthe question is , what happens during those 45 days ? \nwe will need to support elections . \nthere is not a single member of this house who has not supported some form of general election , a special election , to replace the members at some point . \nbut during that 45 days , what happens ? \nthe chair of the constitution subcommittee says this is what happens : martial law . \nwe do not know who would fill the vacancy of the presidency , but we do know that the succession act most likely suggests it would be an unelected person . \nthe sponsors of the bill before us today insist , and i think rightfully so , on the importance of elections . \nbut to then say that during a 45-day period we would have none of the checks and balances so fundamental to our constitution , none of the separation of powers , and that the presidency would be filled

In [8]:
count_vectorizer = CountVectorizer(stop_words='english')
x = count_vectorizer.fit_transform(list_of_speeches)
print(count_vectorizer.get_feature_names())

['000', '018', '10', '100', '108th', '109th', '11', '11th', '12', '122', '123', '125', '13', '135', '14', '14th', '15', '153', '155', '159', '16', '160', '17', '170', '1787', '17th', '18th', '1980s', '1987', '1989', '1999', '19th', '20', '200', '2001', '2002', '2004', '2005', '2006', '20th', '21', '21st', '2217', '23', '234', '23rd', '24', '25', '2586', '26', '2646', '26th', '27', '270', '28', '2882', '2884', '2888', '2904', '2926', '2944', '2975', '2985', '30', '3004', '306', '3061', '3090', '3160', '3162', '317', '32', '329', '33', '345', '35', '353', '36', '365', '40', '435', '44', '45', '47', '479', '49', '50', '52', '521', '53', '539', '543', '55', '57', '5th', '60', '62', '63', '64', '65', '650', '653', '68', '692', '698', '70', '754', '84', '841', '868', '87', '870', '97', '99', 'abandoned', 'ability', 'able', 'abraham', 'abridgement', 'abrogation', 'absence', 'absentee', 'absolutely', 'acceptable', 'access', 'accessible', 'accident', 'accidents', 'accomplish', 'accomplished', '

In [9]:
pd.DataFrame(x.toarray(), columns=count_vectorizer.get_feature_names())

,000,018,10,100,108th,109th,11,11th,12,122,123,125,13,135,14,...,xz4003650,xz4003770,xz4004200,xz4004220,xz4004240,xz4006261,year,years,yeas,yes,yesterday,yield,yielded,yielding,york
0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Okay, it's **far** too big to even look at. Let's try to get a list of features from a new `CountVectorizer` that only takes the top 100 words.

In [10]:
count_vectorizer = CountVectorizer(stop_words='english', max_features=100)
x = count_vectorizer.fit_transform(list_of_speeches)
print(count_vectorizer.get_feature_names())

['11', '2001', '45', 'able', 'act', 'address', 'allow', 'amendment', 'american', 'appropriations', 'ask', 'balance', 'believe', 'body', 'branch', 'california', 'capitol', 'chairman', 'citizens', 'colleagues', 'committee', 'congress', 'constitution', 'constitutional', 'continuity', 'country', 'day', 'days', 'debate', 'democracy', 'distinguished', 'does', 'elected', 'election', 'elections', 'event', 'fact', 'forward', 'funding', 'gentleman', 'going', 'good', 'government', 'great', 'having', 'history', 'horses', 'house', 'important', 'institution', 'issue', 'just', 'know', 'law', 'legislation', 'legislative', 'let', 'make', 'member', 'members', 'million', 'mounted', 'mr', 'need', 'opportunity', 'order', 'people', 'percent', 'point', 'police', 'president', 'process', 'representation', 'representatives', 'right', 'rule', 'rules', 'said', 'say', 'senate', 'september', 'simply', 'speaker', 'special', 'state', 'states', 'support', 'terrorists', 'think', 'time', 'today', 'united', 'vote', 'vote

Now let's push all of that into a dataframe with nicely named columns.

In [18]:
pd.DataFrame(x.toarray(), columns=count_vectorizer.get_feature_names())

,11,2001,45,able,act,address,allow,amendment,american,appropriations,ask,balance,believe,body,branch,...,states,support,terrorists,think,time,today,united,vote,voters,want,way,wisconsin,work,year,yield
0,1,0,7,0,3,1,0,0,3,0,2,0,1,6,0,...,4,1,0,3,3,2,1,0,0,1,1,1,0,0,1
1,0,0,1,1,1,0,1,1,0,0,0,1,0,0,0,...,0,1,6,0,2,2,0,1,0,1,3,0,0,1,0
2,0,0,3,0,0,1,0,0,1,0,0,0,0,1,1,...,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,...,1,0,1,0,2,0,1,1,0,1,1,0,0,0,0
4,0,0,0,0,0,4,0,1,0,1,1,0,0,0,0,...,0,0,0,0,1,0,0,2,0,0,0,0,0,0,2
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
6,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,...,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,1,0,1,1,1,0,0,0,0,...,1,0,0,2,0,0,1,0,0,0,0,0,0,0,0


Everyone seems to start their speeches with "mr chairman" - how many speeches are there total, and many don't mention "chairman" and how many mention neither "mr" nor "chairman"?

What is the index of the speech thank is the most thankful, a.k.a. includes the word 'thank' the most times?

If I'm searching for `China` and `trade`, what are the top 3 speeches to read according to the `CountVectoriser`?

Now what if I'm using a `TfidfVectorizer`?

**What's the content of the speeches?** Here's a way to get them:

In [ ]:
# index 0 is the first speech, which was the first one imported.
paths[0]

In [ ]:
# Pass that into 'cat' using { } which lets you put variables in shell commands
# that way you can pass the path to cat
!cat {paths[0]}

**Now search for something else!** Another two terms that might show up. `elections` and `chaos`? Whatever you thnik might be interesting.

# Enough of this garbage, let's cluster

Using a **simple counting vectorizer**, cluster the documents into **eight categories**, telling me what the top terms are per category.

Using a **term frequency vectorizer**, cluster the documents into **eight categories**, telling me what the top terms are per category.

Using a **term frequency inverse document frequency vectorizer**, cluster the documents into **eight categories**, telling me what the top terms are per category.

In [13]:
from sklearn.cluster import KMeans

number_of_clusters = 8
km = KMeans(n_clusters=number_of_clusters)
km.fit(x)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=8, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [14]:
print("Top terms per cluster:")
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(number_of_clusters):
    top_ten_words = [terms[ind] for ind in order_centroids[i, :5]]
    print("Cluster {}: {}".format(i, ' '.join(top_ten_words)))

Top terms per cluster:


NameError: name 'vectorizer' is not defined

**Which one do you think works the best?**

# Harry Potter time

I have a scraped collection of Harry Potter fanfiction at https://github.com/ledeprogram/courses/raw/master/algorithms/data/hp.zip.

I want you to read them in, vectorize them and cluster them. Use this process to find out **the two types of Harry Potter fanfiction**. What is your hypothesis?

In [23]:
paths = glob.glob('hp/10196712.txt*')
paths[:5]

['hp/10196712.txt']

In [22]:
len(paths)

1